In [2]:
%load_ext autoreload

In [3]:
%autoreload

import os, random, pandas as pd, numpy as np
from sklearn.model_selection import StratifiedKFold
import pickle
import sys
sys.path.append('../..')
import relation_extraction.data.utils as utils
import itertools
# SEED = 1
# np.random.seed(SEED)
# random.seed(SEED)
RESOURCE_PATH = "/data/medg/misc/semeval_2010"
def res(path): return os.path.join(RESOURCE_PATH, path)

[nltk_data] Downloading package wordnet to
[nltk_data]     /afs/csail.mit.edu/u/g/geeticka/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [58]:
train_data = open(res('train.txt'))
splitted_data_border1 = utils.split_data_cut_sentence(train_data, border_size=1)

In [59]:
train_data=open(res('train.txt'))
splitted_data_border20 = utils.split_data_cut_sentence(train_data, border_size=20)

In [60]:
train_data = open(res('train.txt'))
splitted_data_border50 = utils.split_data_cut_sentence(train_data, border_size=50)

In [23]:
train_data = open(res('train.txt'))
splitted_data_border_minus1 = utils.split_data_cut_sentence(train_data)

In [24]:
def shortest_dependency_path(data):
    sentences = data[0]
    relations = data[1]
    e1_pos = data[2]
    e2_pos = data[3]
    paths = []
    paths_with_edge_names = []
    paths_e1_pos = []
    paths_e2_pos = []
    for idx, (sent, pos1, pos2, rel) in enumerate(zip(sentences, e1_pos, e2_pos, relations)):
        path, _, path_with_edge_names = \
        utils.get_shortest_dependency_path(sent, pos1, pos2)
        if path is not None:
            oldnums = [utils.get_only_number(word) for word in path]
            path = [utils.get_only_word(word) for word in path]
            path_pos1, path_pos2 = utils.get_new_entity_position(path, pos1, pos2, oldnums)
        else:
            path = None
            path_pos1 = None
            path_pos2 = None
        paths.append(path)
        paths_e1_pos.append(path_pos1)
        paths_e2_pos.append(path_pos2)
        paths_with_edge_names.append(path_with_edge_names)
    return paths, paths_e1_pos, paths_e2_pos, paths_with_edge_names
        

In [25]:
splitted_data_dependency_borderminus1 = shortest_dependency_path(splitted_data_border_minus1)

In [65]:
splitted_data_dependency_border1 = shortest_dependency_path(splitted_data_border1)

In [ ]:
splitted_data_dependency_border20 = shortest_dependency_path(splitted_data_border20)

In [ ]:
splitted_data_dependency_border50 = shortest_dependency_path(splitted_data_border50)

In [26]:
def get_dataframe(splitted_data, splitted_data_dependency):
    # now need to figure out how to create the csv file
    sen = splitted_data[0]
    rel = splitted_data[1]
    e1 = splitted_data[2]
    e2 = splitted_data[3]
    path = splitted_data_dependency[0]
    path_e1 = splitted_data_dependency[1]
    path_e2 = splitted_data_dependency[2]
    path_edge = splitted_data_dependency[3]
    data = pd.DataFrame({'sentences': sen, 'relations': rel, 'e1_pos': e1, 'e2_pos': e2, 'paths': path, \
                         'paths_e1_pos': path_e1, 'paths_e2_pos': path_e2, 'paths_with_edge_names': path_edge}, \
                        columns=['sentences', 'relations', 'e1_pos', 'e2_pos', 'paths', 'paths_e1_pos', \
                                 'paths_e2_pos', 'paths_with_edge_names'])
    return data

In [27]:
# data_border1 = get_dataframe(splitted_data_border1, splitted_data_dependency_border1)
# data_border20 = get_dataframe(splitted_data_border20, splitted_data_dependency_border20)
# data_border50 = get_dataframe(splitted_data_border50, splitted_data_dependency_border50)
data_borderminus1 = get_dataframe(splitted_data_border_minus1, splitted_data_dependency_borderminus1)

In case you want to rebuild old data:

Make sure that data sizes are correct

In [113]:
#print(len(sen), len(rel), len(e1), len(e2), len(path), len(path_e1), len(path_e2), len(path_edge))

8000 8000 8000 8000 8000 8000 8000 8000


In [114]:
#data

,sentences,relations,e1_pos,e2_pos,paths,paths_e1_pos,paths_e2_pos,paths_with_edge_names
0,"[the, system, as, described, above, has, its, ...",0,"(12, 12)","(15, 15)","[configuration, of, elements]","(0, 0)","(2, 2)","[configuration, pobj_0, of, prep_0, elements]"
1,"[the, child, was, carefully, wrapped, and, bou...",18,"(1, 1)","(9, 9)","[child, wrapped, bound, into, cradle]","(0, 0)","(4, 4)","[child, ROOT_0, wrapped, ROOT_0, bound, conj_0..."
2,"[the, author, of, a, keygen, uses, a, disassem...",1,"(1, 1)","(7, 7)","[author, uses, disassembler]","(0, 0)","(2, 2)","[author, ROOT_0, uses, ROOT_0, disassembler]"
3,"[a, misty, ridge, uprises, from, the, surge, .]",18,"(2, 2)","(6, 6)","[ridge, uprises, from, surge]","(0, 0)","(3, 3)","[ridge, ROOT_1, uprises, ROOT_0, from, prep_0,..."
4,"[the, student, association, is, the, voice, of...",2,"(1, 1)","(2, 2)","[student, association]","(0, 0)","(1, 1)","[student, nsubj_1, association]"
5,"[this, is, the, sprawling, complex, that, is, ...",18,"(4, 4)","(10, 10)","[complex, is, producer]","(0, 0)","(2, 2)","[complex, attr_0, is, relcl_0, producer]"
6,"[the, current, view, is, that, the, chronic, i...",3,"(7, 7)","(19, 19)","[inflammation, in, part, of, stomach, caused, ...","(0, 0)","(8, 8)","[inflammation, attr_0, in, prep_0, part, pobj_..."
7,"[people, have, been, moving, back, into, downt...",4,"(0, 0)","(6, 6)","[people, moving, back, into, downtown]","(0, 0)","(4, 4)","[people, ROOT_1, moving, ROOT_0, back, advmod_..."
8,"[the, lawsonite, was, contained, in, a, platin...",5,"(1, 1)","(6, 7)","[lawsonite, contained, in, crucible]","(0, 0)","(3, 3)","[lawsonite, ROOT_0, contained, ROOT_0, in, pre..."
9,"[the, solute, was, placed, inside, a, beaker, ...",4,"(12, 12)","(20, 20)","[solvent, of, ml, pipetted, into, flask]","(0, 0)","(5, 5)","[solvent, prep_1, of, nsubjpass_1, ml, conj_1,..."


this is how the old data without the directions looks

In [59]:
# data

,sentences,relations,e1_pos,e2_pos,paths,paths_e1_pos,paths_e2_pos,paths_with_edge_names
0,"[the, system, as, described, above, has, its, ...",0,"(12, 12)","(15, 15)","[configuration, of, elements]","(0, 0)","(2, 2)","[configuration, pobj, of, prep, elements]"
1,"[the, child, was, carefully, wrapped, and, bou...",18,"(1, 1)","(9, 9)","[child, wrapped, bound, into, cradle]","(0, 0)","(4, 4)","[child, ROOT, wrapped, ROOT, bound, conj, into..."
2,"[the, author, of, a, keygen, uses, a, disassem...",1,"(1, 1)","(7, 7)","[author, uses, disassembler]","(0, 0)","(2, 2)","[author, ROOT, uses, ROOT, disassembler]"
3,"[a, misty, ridge, uprises, from, the, surge, .]",18,"(2, 2)","(6, 6)","[ridge, uprises, from, surge]","(0, 0)","(3, 3)","[ridge, ROOT, uprises, ROOT, from, prep, surge]"
4,"[the, student, association, is, the, voice, of...",2,"(1, 1)","(2, 2)","[student, association]","(0, 0)","(1, 1)","[student, nsubj, association]"
5,"[this, is, the, sprawling, complex, that, is, ...",18,"(4, 4)","(10, 10)","[complex, is, producer]","(0, 0)","(2, 2)","[complex, attr, is, relcl, producer]"
6,"[the, current, view, is, that, the, chronic, i...",3,"(7, 7)","(19, 19)","[inflammation, in, part, of, stomach, caused, ...","(0, 0)","(8, 8)","[inflammation, attr, in, prep, part, pobj, of,..."
7,"[people, have, been, moving, back, into, downt...",4,"(0, 0)","(6, 6)","[people, moving, back, into, downtown]","(0, 0)","(4, 4)","[people, ROOT, moving, ROOT, back, advmod, int..."
8,"[the, lawsonite, was, contained, in, a, platin...",5,"(1, 1)","(6, 7)","[lawsonite, contained, in, crucible]","(0, 0)","(3, 3)","[lawsonite, ROOT, contained, ROOT, in, prep, c..."
9,"[the, solute, was, placed, inside, a, beaker, ...",4,"(12, 12)","(20, 20)","[solvent, of, ml, pipetted, into, flask]","(0, 0)","(5, 5)","[solvent, prep, of, nsubjpass, ml, conj, pipet..."


In [12]:
# data = pd.read_csv(
#     res('train.txt'),
#     sep=r'\n',
#     names=['Unsplit Sentence'],
#     header=None,
# )
# data['Relation Type'] = data.apply(lambda row: row['Unsplit Sentence'].split()[0], axis=1)

In [28]:
def pickle_data(data, filename1, filename2, SEED):
    np.random.seed(SEED)
    random.seed(SEED)
    print("seed is", SEED)
    N = len(data)
    K = 10

    splitter = StratifiedKFold(n_splits=K, shuffle=True, random_state=SEED)
    fold_indices = [indices for _, indices in splitter.split(
        data[['sentences', 'e1_pos', 'e2_pos', 'paths', 'paths_e1_pos', 'paths_e2_pos', \
              'paths_with_edge_names']].values,
        data['relations'].values
    )]

    assert len(fold_indices) == K, "Incorrect number of folds."
    assert len(np.concatenate(fold_indices)) == N, "Folds not comprehensive."

    splits = []
    for fold in range(K):
        test_fold  = fold_indices[fold]
        dev_fold   = fold_indices[(fold + 1) % K]

        non_train_start = fold
        non_train_end   = (fold + 2) % K

        if non_train_start < non_train_end:
            train_fold = np.concatenate(fold_indices[non_train_end:] + fold_indices[:non_train_start])
        else: train_fold = np.concatenate(fold_indices[non_train_end:non_train_start])
        joined = np.concatenate((train_fold, dev_fold, test_fold))

        assert len(joined) == N, (
            "Split not comprehensive for fold {fold}:\n"
            "  len(train_fold): {train}\n"
            "  len(dev_fold): {dev}\n"
            "  len(test_fold): {test}\n"
            "  len(joined): {joined}\n"
            "  N: {N}".format(
                N=N, fold=fold, train=len(train_fold), dev=len(dev_fold), test=len(test_fold), joined=len(joined)
            )
        )

        splits.append((
            data.iloc[train_fold],
            data.iloc[dev_fold],
            data.iloc[test_fold],
        ))

    assert len(splits) == K, "Too few splits"

    #below has the dependency info with the directionalities
    #seed_1_{K}-dep-dir-fold.pkl
    with open(res('pickled-files/'+filename1.format(K=K)), mode='wb') as f: pickle.dump(splits, f)
    with open(res('pickled-files/'+filename2.format(K=K)), mode='wb') as f: pickle.dump(splits, f, protocol=2)
    #seed_1_{K}-dep-dir-fold_py2.pkl
    # below has the dependency info without the directionalities
    # with open(res('seed_1_{K}-dep-fold.pkl'.format(K=K)), mode='wb') as f: pickle.dump(splits, f)
    # with open(res('seed_1_{K}-dep-fold_py2.pkl'.format(K=K)), mode='wb') as f: pickle.dump(splits, f, protocol=2)
    # dumping old data
    # with open(res('seed_1_{K}-fold.pkl'.format(K=K)), mode='wb') as f: pickle.dump(splits, f)
    # with open(res('seed_1_{K}-fold_py2.pkl'.format(K=K)), mode='wb') as f: pickle.dump(splits, f, protocol=2)

In [29]:
for seed in [1,5]:
    str_seed = str(seed)
    pickle_data(data_borderminus1, 'seed_'+str_seed+'_{K}-dep-dir-fold-border_-1.pkl', 'seed_'+str_seed+'_{K}-dep-dir-fold-border_-1_py2.pkl', seed)

seed is 1


/scratch/conda_envs/ontology-aided-relex/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


seed is 5


In [89]:
# pickle_data(data_border1, 'seed_1_{K}-dep-dir-fold-border_1.pkl', 'seed_1_{K}-dep-dir-fold-border_1_py2.pkl')
# pickle_data(data_border20, 'seed_1_{K}-dep-dir-fold-border_20.pkl', 'seed_1_{K}-dep-dir-fold-border_20_py2.pkl')
# pickle_data(data_border50, 'seed_1_{K}-dep-dir-fold-border_50.pkl', 'seed_1_{K}-dep-dir-fold-border_50_py2.pkl')

/scratch/conda_envs/ontology-aided-relex/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


In [49]:
# pickle_data(data_border50, 'seed_1_{K}-dep-dir-fold-border_50.pkl', 'seed_1_{K}-dep-dir-fold-border_50_py2.pkl', 1)
# pickle_data(data_border50, 'seed_2_{K}-dep-dir-fold-border_50.pkl', 'seed_2_{K}-dep-dir-fold-border_50_py2.pkl', 2)
# pickle_data(data_border50, 'seed_3_{K}-dep-dir-fold-border_50.pkl', 'seed_3_{K}-dep-dir-fold-border_50_py2.pkl', 3)
# pickle_data(data_border50, 'seed_4_{K}-dep-dir-fold-border_50.pkl', 'seed_4_{K}-dep-dir-fold-border_50_py2.pkl', 4)
# pickle_data(data_border50, 'seed_5_{K}-dep-dir-fold-border_50.pkl', 'seed_5_{K}-dep-dir-fold-border_50_py2.pkl', 5)

seed is 1


/scratch/conda_envs/ontology-aided-relex/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


seed is 2
seed is 3
seed is 4
seed is 5


In [ ]:
pickle_data(data_borderminus1, 'seed_5_{K}-dep-dir-fold-border_-1.pkl', 'seed_5_{K}-dep-dir-fold-border_-1_py2.pkl', 5)
pickle_data(data_border1, 'seed_5_{K}-dep-dir-fold-border_1.pkl', 'seed_5_{K}-dep-dir-fold-border_1_py2.pkl', 5)
pickle_data(data_border20, 'seed_5_{K}-dep-dir-fold-border_20.pkl', 'seed_5_{K}-dep-dir-fold-border_20_py2.pkl', 5)
pickle_data(data_border50, 'seed_5_{K}-dep-dir-fold-border_50.pkl', 'seed_5_{K}-dep-dir-fold-border_50_py2.pkl', 5)

In [50]:
TRAIN, DEV, TEST = 0, 1, 2
class Dataset():
    def __init__(self, relations_split_file):
        with open(relations_split_file, mode='rb') as f: self.relations_splits = pickle.load(f)
        self.K = len(self.relations_splits)

    def get_data_for_fold(self, fold_num, data_type=TRAIN):
        assert fold_num < self.K
        data = self.relations_splits[fold_num][data_type]
        return data['sentences'].tolist(), data['relations'].tolist(), data['e1_pos'].tolist(), \
    data['e2_pos'].tolist(), data['paths'].tolist(), data['paths_e1_pos'].tolist(), \
    data['paths_e2_pos'].tolist(), data['paths_with_edge_names'].tolist()
        # we need it in list format

    def get_full_data(self):
        data = pd.concat([self.relations_splits[0][t] for t in [DEV, TEST, TRAIN]])
        return data.tolist()

## Just to double check, see if the lengths of the datasets make sense

### No sentence in category of border size 1 should be larger than a sentence in category of border size 50

In [6]:
dataset50 = Dataset(res('pickled-files/seed_1_10-dep-dir-fold-border_50.pkl'))
dataset1 = Dataset(res('pickled-files/seed_1_10-dep-dir-fold-border_1.pkl'))
datasetminus1 = Dataset(res('pickled-files/seed_1_10-dep-dir-fold-border_-1.pkl'))
dataset20 = Dataset(res('pickled-files/seed_1_10-dep-dir-fold-border_20.pkl'))

In [7]:
data50 = dataset50.get_data_for_fold(0, TEST)
data1 = dataset1.get_data_for_fold(0, TEST)
dataminus1 = datasetminus1.get_data_for_fold(0, TEST)
data20 = dataset20.get_data_for_fold(0, TEST)

In [10]:
def list_of_elts(data):
    elts = [len(x) for x in data]
    return elts

elts50 = list_of_elts(data50[0])
elts1 = list_of_elts(data1[0])
eltsminus1 = list_of_elts(dataminus1[0])
elts20 = list_of_elts(data20[0])

In [12]:
def compare_lengths(smaller, bigger):
    for idx, (small, big) in enumerate(zip(smaller, bigger)):
        if(small > big):
            print("Smaller list has a bigger length than bigger list!")
            break

In [21]:
compare_lengths(elts1, elts50)
compare_lengths(elts1, eltsminus1)
compare_lengths(elts1, elts20)

In [22]:
compare_lengths(elts50, eltsminus1)

In [23]:
compare_lengths(elts20, elts50)
compare_lengths(elts20, eltsminus1)

Hypothesis is correct; border size parameter is behaving correctly

In [69]:
# usage info for old data without dependency and splitting information
# TRAIN, DEV, TEST = 0, 1, 2
# class Dataset():
#     def __init__(self, relations_split_file):
#         with open(relations_split_file, mode='rb') as f: self.relations_splits = pickle.load(f)
#         self.K = len(self.relations_splits)

#     def get_data_for_fold(self, fold_num, data_type=TRAIN):
#         assert fold_num < self.K
#         data = self.relations_splits[fold_num][data_type]
#         return data['Unsplit Sentence'].tolist()
#         # we need it in list format

#     def get_full_data(self):
#         data = pd.concat([self.relations_splits[0][t] for t in [DEV, TEST, TRAIN]])
#         return data.tolist()

In [70]:
# dataset = Dataset(res('seed_1_10-fold.pkl'))

In [71]:
# dataset.get_data_for_fold(9, TEST)

# Check if Seeds are actually creating different data

In [51]:
dataset_seed1 = Dataset(res('seed_1_10-dep-dir-fold-border_50.pkl'))
dataset_seed2 = Dataset(res('seed_2_10-dep-dir-fold-border_50.pkl'))
dataset_seed3 = Dataset(res('seed_3_10-dep-dir-fold-border_50.pkl'))
dataset_seed4 = Dataset(res('seed_4_10-dep-dir-fold-border_50.pkl'))
dataset_seed5 = Dataset(res('seed_5_10-dep-dir-fold-border_50.pkl'))

In [52]:
seed1dev = dataset_seed1.get_data_for_fold(8, DEV)
seed2dev = dataset_seed2.get_data_for_fold(8, DEV)
seed3dev = dataset_seed3.get_data_for_fold(8, DEV)
seed4dev = dataset_seed4.get_data_for_fold(8, DEV)
seed5dev = dataset_seed5.get_data_for_fold(8, DEV)

In [53]:
def compare_list_of_list(lista, listb):
    lista_set = set(map(tuple, lista))
    listb_set = set(map(tuple, listb))
    return lista_set.symmetric_difference(listb_set)

In [54]:
compare_things = [seed1dev[0], seed2dev[0], seed3dev[0], seed4dev[0], seed5dev[0]]

In [55]:
for pair in itertools.combinations(compare_things, r=2):
    print(compare_list_of_list(*pair))

{('all', 'parties', 'were', 'taken', 'into', 'confidence', 'over', 'swat', 'offensive', '.'), ('the', 'infamous', 'teacher', 'has', 'been', 'dragged', 'into', 'another', 'child', 'abuse', 'case', '.'), ('the', 'program', 'assists', 'separating', 'families', 'in', 'high', 'conflict', 'over', 'parenting', 'arrangements', '.'), ('new', 'experiences', 'throw', 'neurons', 'into', 'reverse', '.'), ('this', 'is', 'the', 'best', 'armor', 'for', 'knights', '.'), ('the', 'winner', 'has', 'received', 'the', 'years', 'worth', 'of', 'beauty', 'products', 'in', 'a', 'hamper', '.'), ('the', 'activities', 'are', 'divided', 'into', 'three', 'programs', ':', 'the', 'children', 'and', 'youth', 'programme', ',', 'the', 'adult', 'programme', 'and', 'the', 'senior', 'citizens', 'programme', '.'), ('the', 'bottom', 'photo', 'is', 'from', 'the', 'new', 'york', 'public', 'library', '.'), ('however', ',', 'the', 'increased', 'demand', 'logically', 'calls', 'for', 'expansion', 'and', 'modification', 'of', 'intak

means things are ok after setting shuffle to True